# Guía 9 - Pronósticos de demanda diaria con LSTM usando Keras y TensorFlow

## Pronósticos a partir de la serie temporal de la demanda diaria incorporando variables exógenas desde Datasets

La propuesta es dar pronósticos de la demanda diaria de energía eléctrica a partir de sus valores registrados en días anteriores y de otros atributos que podrían tener influencia (variables exógenas) alamcenados en `tf.data.Dataset`. 

Continuaremos con nuestro ejemplo tomando la base de datos de demanda diaria de energía eléctrica publicados por CAMMESA. 

## Inicio de la programación para el análisis y pronóstico de la serie temporal

Comenzamos importando las bibliotecas necesarias y confirmamos la versión de TensorFlow (TF) disponible en la instalación. Esto nos puede ayudar a interpretar algunas diferencias en el comportamiento de las funciones de TF.

In [ ]:
import datetime
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error

import keras
import tensorflow as tf
tf.random.set_seed(99)
print(tf.__version__)

## Carga del conjunto de datos para su análisis

Cargamos los datos de demanda diaria. DEMANDA TOTAL es la serie que queremos predecir (target). Esperamos que sus valores históricos nos provean información para su pronóstico. Además, tomaremos otros atributos que consideramos relevantes como `TEMPERATURA REFERENCIA MEDIA GBA °C` o `Tipo día` que nos informa si la jornada fue laborable, semilaborable o no laborable. Las columnas que se refieren a los registros de demanda para otras regiones argentinas no las consideraremos.

In [ ]:
# Dataset loading
dataFrame = pd.read_excel('Data/Base Demanda Diaria 2017 2024.xlsx', sheet_name='Datos Región', skiprows=4)  
dataFrame.info()

### Separación de los conjuntos de datos de entrenamiento, validación y prueba

Como es habitual, separamos el conjunto de datos en entrenamiento y prueba. Como sabemos, se reserva una porción del final de la serie temporal como datos de prueba. En este ejercicio, desdoblamos en datos numéricos y categóricos.

In [ ]:
orig_data = {}

col_names = {'DEMANDA TOTAL': 'num0_input', 'TEMPERATURA REFERENCIA MEDIA GBA °C': 'num1_input', 'Tipo día': 'cat_input'}
new_df = dataFrame.rename(columns=col_names)

for key in col_names.values():
    if new_df[key].dtype == np.float64:
        orig_data[key] = np.float32(new_df[key].to_numpy())
    else:
        orig_data[key] = new_df[key].to_numpy()

orig_ds = tf.data.Dataset.from_tensor_slices(orig_data)

orig_ds.element_spec

In [ ]:
for element in orig_ds.take(1):
    print(element['num0_input'].shape)
    print(element['num0_input'])

## Alternativas para time series windows

usando `.batch()` y `.window()`

TF: Build TensorFlow input pipelines

[Time series windowing](https://www.tensorflow.org/guide/data#time_series_windowing)

## Ventanas de datos






In [ ]:
SEQ_LENGTH = 28
OUT_LENGTH = 1
BATCH_SIZE = 32

windowed_ds = orig_ds.window(size=SEQ_LENGTH + OUT_LENGTH, shift=1, drop_remainder=True)

# Applying window to a Dataset of dictionaries gives a dictionary of Datasets
#for windows in windowed_ds.take(1):
#  print(tf.nest.map_structure(to_numpy, windows))

new_ds = windowed_ds.flat_map(lambda window: tf.data.Dataset.zip(
    dict([(k, v.batch(SEQ_LENGTH + OUT_LENGTH)) for k, v in window.items()])))

dict_ds = new_ds.map(lambda x: ((x['num0_input'][:-OUT_LENGTH], x['num1_input'][:-OUT_LENGTH],
                                 x['cat_input'][:-OUT_LENGTH]), x['num0_input'][-OUT_LENGTH:]))

print(dict_ds.element_spec, '\n')
dict_ds = dict_ds.map(lambda x,y: ((tf.stack([x[0], x[1]], axis=1), tf.expand_dims(x[2], 1)), y))


dict_ds = dict_ds.batch(BATCH_SIZE, drop_remainder=True)


print(dict_ds.element_spec, '\n')
for a,b in dict_ds.take(1):
    print(a[0].shape, a[1].shape, b.shape)
#    print(a[1][1,:,:])
#    print(b)

Preguntamos si conocemos la cantidad de lotes que conforman el `DATASET

In [ ]:
DATASET_SIZE = dict_ds.cardinality()
print((DATASET_SIZE == tf.data.UNKNOWN_CARDINALITY).numpy())

Como no se sabe la cantidad de lotes que forman el `DATASET`, entonces cargamos manualmente su número mediante un cálculo que considera el tamaño original y el de los lotes...

In [ ]:
DATASET_SIZE = (new_df.shape[0] - SEQ_LENGTH - OUT_LENGTH) // BATCH_SIZE
dict_ds = dict_ds.apply(tf.data.experimental.assert_cardinality(DATASET_SIZE))
print(dict_ds.cardinality().numpy())

Separamos en datos de entrenamiento, validación y prueba

In [ ]:
train_size = int(0.82 * DATASET_SIZE)
val_size = int(0.1 * DATASET_SIZE)

train_ds = dict_ds.take(train_size)
val_ds = dict_ds.skip(train_size).take(val_size)
test_ds = dict_ds.skip(train_size).skip(val_size)


Ahora no es necesario separar en atributos (`X`) y targets (`y`), solo debemos pasar los `Datasets` de entrenamiento o prueba

### Adaptar para normalizar los atributos numéricos 

In [ ]:
# Normalize numeric features
# normalize0_layer = keras.layers.Normalization(axis=0, mean=15208, variance=4289041)
normalize_layer = keras.layers.Normalization(axis=2)
normalize_layer.adapt(train_ds.map(lambda X, y: X[0]))
normalize_layer.get_config()



Podemos conocer la media y la varianza que luego podríamos usar para invertir los resultados

In [ ]:
print(normalize_layer.mean)

In [ ]:
print('DEMANDA TOTAL - Mean: {0:.0f}, Std.Dev {1:.0f}'.format(normalize_layer.mean[0,0,0],
                                                           np.sqrt(normalize_layer.variance[0,0,0])))
print('Temperatura - Mean: {0:.1f}, Std.Dev. {1:.1f}'.format(normalize_layer.mean[0,0,1],
                                                             np.sqrt(normalize_layer.variance[0,0,1])))

### Agregando la capa custom de preprocesado para los atributos categóricos

In [ ]:
# Our definition for the categorical preporcossing layer
class OneHotPre(keras.layers.Layer):
    def __init__(self, max_tokens, vocabulary=None, num_oov_indices=0, **kwargs):
        self._lower_lyr = tf.keras.layers.Lambda(lambda x: tf.strings.lower(x))
        self._lab_lyr = tf.keras.layers.Lambda(lambda x: 
                                           tf.strings.regex_replace(x, '^.*h.bil.*$', 'laborable'))
        self._semilab_lyr = tf.keras.layers.Lambda(lambda x:
                                                   tf.strings.regex_replace(x,'^.*semilaborable.*$', 'semilaborable'))
        self._nolab_lyr = tf.keras.layers.Lambda(lambda x:
                                                 tf.strings.regex_replace(x,'^.*feriado.*$', 'nolaborable'))
        self._lookup_layer = tf.keras.layers.StringLookup(
            max_tokens = max_tokens,
            num_oov_indices = num_oov_indices,
            vocabulary = vocabulary,
            # output_mode = 'int')
            output_mode = 'one_hot')
        super().__init__(**kwargs)
    def _to_day_type(self, value):
        day_type = self._lower_lyr(value)
        day_type = self._lab_lyr(day_type)
        day_type = self._semilab_lyr(day_type)
        return self._nolab_lyr(day_type)
    def adapt(self, data_sample):
        map_ds = data_sample.map(lambda x: self._lower_lyr(x))
        map_ds = map_ds.map(lambda x: self._lab_lyr(x))
        map_ds = map_ds.map(lambda x: self._semilab_lyr(x))
        map_ds = map_ds.map(lambda x: self._nolab_lyr(x))
        self._lookup_layer.adapt(map_ds)
        # self._lookup_layer.adapt(data_sample)
    def call(self, inputs):
        # return self._lookup_layer(self._to_day_type(inputs))
        ll = self._lower_lyr(inputs)
        ll = self._lab_lyr(ll)
        ll = self._semilab_lyr(ll)
        ll = self._nolab_lyr(ll)
        return self._lookup_layer(ll)
    def get_config(self):
        return {'vocabulary': self._lookup_layer.get_vocabulary()}

### Adaptando la capa custom para preprocesar los attributos categóricos

In [ ]:
index=0
one_hot_layer = OneHotPre(3)
one_hot_layer.adapt(train_ds.map(lambda X, y: X[1]))
one_hot_layer.get_config()

#one_hot_layer = OneHotPre(index, 7, ['Domingo o Feriado',
#                                     'Sabado o Semilaborable', 
#                                     'Miercoles habiles', 
#                                     'Jueves Habiles', 
#                                     'Martes Habiles',
#                                     'Viernes habiles',
#                                     'Lunes habiles'])
one_hot_layer.get_config()

Confirmamos los tamaños de los nuevos conjuntos de datos particionados en lotes.

In [ ]:
for batch in train_ds.take(1):
    inputs, outputs = batch
    # outputs = outputs[...,tf.newaxis]
    print('Input len: ', len(inputs), 'Output len:', len(outputs))
    print('Input 0 (num) shape: ', inputs[0].shape)
    print('Input 1 (cat) shape: ', inputs[1].shape)
    print('Ouput shape: ', outputs.shape)
    other0=normalize_layer(inputs[0])
    other2=one_hot_layer(inputs[1])
    print('Normalized shape ', other0.shape)
    print('One-hot shape ', other2.shape)
    print('One-hot type ', other2.dtype)


## Definición del modelo 



En este caso, definimos el modelo de manera que procese la información de entrada por 2 canales diferentes. Los datos numéricos se normalizan, los categóricos pasan por la capa custom de codificación one-hot. Luego, ambas salidas se unen para continuar su procesamiento en la primer capa con celdas LSTM. 


## Como pasar multiples datos de entrada desde un Dataset

TF: Training & evaluation with the built-in methods 

the Dataset should return a tuple of dicts.

[Passing data to multi-input, multi-output models](https://www.tensorflow.org/guide/keras/training_with_built_in_methods#passing_data_to_multi-input_multi-output_models)

y también ver TF: The Functional API

When calling fit with a Dataset object, it should yield either a tuple of lists like `([title_data, body_data, tags_data], [priority_targets, dept_targets])` or a tuple of dictionaries like `({'title': title_data, 'body': body_data, 'tags': tags_data}, {'priority': priority_targets, 'department': dept_targets})`.

[Manipulate complex graph topologies](https://www.tensorflow.org/guide/keras/functional_api#manipulate_complex_graph_topologies)

In [ ]:
unnormalize_layer = keras.layers.Normalization(axis=1, mean=normalize_layer.mean[0,0,0],
                                              variance=normalize_layer.variance[0,0,0],
                                               invert=True, name='output')

# Functional API Model 
numeric_input = keras.layers.Input(shape=(SEQ_LENGTH,2), dtype=tf.float32, name='num_input')
normalized = normalize_layer(numeric_input)
categorical_input = keras.layers.Input(shape=(SEQ_LENGTH,1), dtype=tf.string, name='cat_input')
encoded = one_hot_layer(categorical_input)
reshape_layer = tf.keras.layers.Reshape((SEQ_LENGTH,3))(encoded)
concat = keras.layers.concatenate([normalized, reshape_layer])
lstm_lyr = keras.layers.LSTM(10)(concat)
dropout_lyr = keras.layers.Dropout(0.2)(lstm_lyr)
dense_lyr = keras.layers.Dense(1, activation='linear')(dropout_lyr)
unnormalized = unnormalize_layer(dense_lyr)
model_exo = keras.models.Model(inputs=[numeric_input, categorical_input], outputs=[unnormalized])
model_exo.compile(loss = 'MeanSquaredError', metrics=['MAE'], optimizer='Adam')
# model_exo.compile(loss = {'output': tf.keras.losses.MeanSquaredError()},
#                  metrics={'output': tf.keras.metrics.MeanAbsoluteError()},
#                  optimizer='Adam')
model_exo.summary()

El método `.summary()` nos muestra cómo fluirá la información entre las capas de NN y cuántos parámetros se deberán ajustar en cada una. Otra alternativa para representar la configuración de la NN es usar `tf.keras.utils.plot_model`.

In [ ]:
tf.keras.utils.plot_model(model_exo, to_file='Figs/model.png', rankdir="TB", show_shapes=True,  show_layer_names=True)

#tf.keras.utils.plot_model(multivariate_lstm, to_file='Figs/model.png', show_shapes=True,
#                          show_dtype=True, show_layer_names=True, rankdir='TB',
#                          expand_nested=True, dpi=200, show_layer_activations=True,
#                          show_trainable=True)

### Ajuste del modelo

Entrenamos el modelo, es decir, ajustamos los pesos de acuerdo al esquema definido cuando fue compilado, por ejemplo optimizer='Adam'

In [ ]:
# Model training
history = model_exo.fit(train_ds, epochs=30, validation_data=val_ds)

# early_stopping_cb = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
# history = model_exo.fit(train_ds, epochs=3000, validation_data=val_ds, callbacks=[early_stopping_cb])

Para tener una idea sobre la convergencia del moelo, mostramos las curvas de aprendizaje

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.yscale('log')
plt.grid(True)
# plt.gca().set_ylim(0,1)

Luego que el modelo ha sido ajustado, es muy fácil obtener nuevas predicciones, como por ejemplo cuando le pasamos el conjunto de prueba.

In [ ]:

# Predict discards targets
predicted_values = model_exo.predict(test_ds)

In [ ]:
predicted_values.shape

In [ ]:
#target_values = test_ds.map(lambda X, y: y['output'])
#target_values = target_values.flat_map(tf.data.Dataset.from_tensor_slices)
#print(type(target_values))
#target_values = np.fromiter(target_values.as_numpy_iterator(), dtype=np.float32)
#print(target_values.shape)

# or
target_values = test_ds.map(lambda X, y: y)
target_values = target_values.unbatch()
print(type(target_values))
target_values = list(target_values.as_numpy_iterator())
print(len(target_values))

### Evaluación de los resultados

Ahora vamos a representar nuestro resultado. Construimos un `DataFrame` para almacenar los valores de `DEMANDA TOTAL` predecida y observada, y su fecha (que recuperamos de los datos originales). Este `DataFrame` nos facilitará el cálculo de distintas métricas que califican el grado de acuerdo de las predicciones del modelo con los valores registrados. Asimismo, representaremos en una figura ambas series de datos para tener una estimación visual del desempeño del modelo.  

In [ ]:
# Forecast Plot with Dates on X-axis
LSTM_eval = pd.DataFrame({
    'Predicted_DEMANDA': predicted_values[:, 0],
    'Actual_DEMANDA': target_values,
})

LSTM_eval.loc[:, 'Date'] = dataFrame['Fecha'][-len(target_values):].values
# LSTM_eval.set_index('Date', inplace=True)
LSTM_eval['Date'] = pd.to_datetime(LSTM_eval['Date'])
LSTM_eval.set_index('Date', inplace=True)


In [ ]:
# Creamos una figura para representar las series de datos
def FigPredActual(d, title):
    fig, ax = plt.subplots(figsize=(10, 6))
    #  highlight the  forecast
    # highlight_start = int(len(d) * 0.9)  
    # highlight_end = len(d) - 1  # Adjusted to stay within bounds
    # Plot the actual values
    # plt.plot(d[['Actual_DEMANDA']][:highlight_start], label=['Actual_DEMANDA'])
    plt.plot(d[['Actual_DEMANDA']], label=['Actual_DEMANDA'])
    
    # Plot predicted values with a dashed line
    plt.plot(d[['Predicted_DEMANDA']], label=['Predicted_DEMANDA'], linestyle='--')
    
    # Highlight the forecasted portion with a different color
    # plt.axvspan(d.index[highlight_start], d.index[highlight_end], facecolor='lightgreen', alpha=0.5, label='Forecast')
    
    plt.title(title)
    plt.xlabel('Dates')
    plt.ylabel('Values')
    ax.legend()

Trazamos ambas series de datos

In [ ]:
FigPredActual(LSTM_eval, 'Multivariate Time-Series forecasting using LSTM and Exogenous variables')

Definimos una función conveniente para calcular diferentes métrias que nos informan sobre el grado de acuerdo de las predicciones del modelo con las observaciones

In [ ]:
# Model Evaluation
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def performance(d):
    return {
        'MSE': mean_squared_error(d['Actual_DEMANDA'].to_numpy(), d['Predicted_DEMANDA'].to_numpy()),
        'MAE': mean_absolute_error(d['Actual_DEMANDA'].to_numpy(), d['Predicted_DEMANDA'].to_numpy()),
        'R2': r2_score(d['Actual_DEMANDA'].to_numpy(), d['Predicted_DEMANDA'].to_numpy())
    }

performance(LSTM_eval)

**Preguntas**

1. ¿Mejora el ajuste si `BATCH_SIZE` y `SEQ_LENGTH` coinciden?
2. ¿Es importante que `SEQ_LENGTH` tome una cantidad de días que corresponda a un número entero de semanas?

**Ejercicio**

1. Ajustar la serie temporal de `DEMANDA TOTAL` usando solamente la variable endógena construtendo los `DATASETS` de la siguiente manera:

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(dataFrame['DEMANDA TOTAL'])
dataset = dataset.window(SEQ_LENGTH + OUT_LENGTH, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(SEQ_LENGTH + OUT_LENGTH))
dataset = dataset.map(lambda window: (window[:-OUT_LENGTH], window[-OUT_LENGTH:]))

¿Se podría repetir esta técnica sobre otros atributos para incluir variables exógenas, como una alternativa al uso de diccionarios?

Para más información ver también:

* [How to use windows created by the Dataset.window() method in TensorFlow 2.0?](https://stackoverflow.com/questions/55429307/how-to-use-windows-created-by-the-dataset-window-method-in-tensorflow-2-0)
* tf.data: Build TensorFlow input pipelines [Time series windowing](https://www.tensorflow.org/guide/data#time_series_windowing)

2. Ajustar la serie temporal de `DEMANDA TOTAL` usando solamente la variable endógena construyendo los `DATASETS` de la siguiente manera:

In [ ]:
def make_dataset(data, target_idx=0, out_len=28, ts_num=28):
    input_data = data[:-ts_num]
    if target_idx is not None:
        targets = data[ts_num:, target_idx]
        targets = targets[...,np.newaxis]
    else:
        targets = None
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=input_data,
        targets=targets,
        sequence_length=out_len, #Length of the output sequences (in number of timesteps). 
        sequence_stride=1, #Period between successive output sequences
        shuffle=False,
        batch_size=ts_num, #Number of timeseries samples in each batch
        seed=43)
    return ds

# num_attr_ds = make_dataset(dataFrame['DEMANDA TOTAL'], 0, SEQ_LENGTH, BATCH_SIZE)

`tf.keras.preprocessing.timeseries_dataset_from_array` toma una secuencia de datos reunidos en intervalos iguales, junto con parámetros de la serie temporal como longitud de la secuencia/ventana, espaciamiento de la secuencia/ventana, etc., para producir lotes de series temporales de datos de entrada y de de targets.

¿Se podría repetir esta técnica sobre otros atributos para incluir variables exógenas, como una alternativa al uso de diccionarios?

Para más información ver también:
* [timeseries_dataset_from_array](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/timeseries_dataset_from_array)
* Un guía completa sobre como armar distintas ventanas de datos usando `timeseries_dataset_from_array` se enecuentra en [Time series forecasting](https://www.tensorflow.org/tutorials/structured_data/time_series). El enlace es una referencia al uso de NN para pronósticos de series temporales 
